In [1]:
from music21 import *
import os 
import numpy as np
import matplotlib.pyplot as plt
from Utilities.midi_helper import *
import matplotlib.pyplot as plt
import Generate_dataset

In [2]:
#specify the path
path='./schubert/'

#read all the filenames
files=[i for i in os.listdir(path) if i.endswith(".mid")]

#reading each midi file
notes_array = np.array([read_midi(path+files[0])])
# notes_array = np.array([read_midi(path+i) for i in files])

Loading Music File: ./schubert/schub_d760_1.mid


In [3]:
x_seq , y_seq , unique_notes,note_to_int  = Generate_dataset.get_sequences(notes_array,32)

**Testing the Model by training sinosoids**



*   Uncomment the code chunk below to test o sinosoids 
*   dont run above code if training for sinosoids. Variable names are mostly same



In [ ]:
time = np.linspace(1,10,10000)
# print(time)
y = np.sin(20*time)
# plt.figure();
# plt.plot(y)
func = np.round(y,decimals=2)
# print(y[:100])
no_of_timesteps = 32
x = []
y = []


for i in range(0, len(func) - no_of_timesteps-1, 1):
    
    #preparing input and output sequences
    input_ = func[i:i + no_of_timesteps]
    output = func[i+no_of_timesteps+1]
    
    x.append(input_)
    y.append(output)
        
x=np.array(x)
y=np.array(y)

unique_x = list(set(x.ravel()))
x_note_to_int = dict((note_, number) for number, note_ in enumerate(unique_x))
x_seq=[]
for i in x:
    temp=[]
    for j in i:
        #assigning unique integer to every note
        temp.append(x_note_to_int[j])
    x_seq.append(temp)
    
x_seq = np.array(x_seq)

unique_y = list(set(y))
y_note_to_int = dict((note_, number) for number, note_ in enumerate(unique_y)) 
y_seq=np.array([x_note_to_int[i] for i in y])


In [4]:
import torch
from torch import tensor
import torch.nn as nn
import torch.optim as optim
import tensorflow as tf
import torch.nn.functional as F
import math

In [5]:
from MusicDataset import *

In [6]:
from sklearn.model_selection import train_test_split
x_tr, x_val, y_tr, y_val = train_test_split(x_seq,y_seq,test_size=0.2,random_state=0)

In [7]:
batch_size = 8
train_set = MusicDataset(x_tr,y_tr)
trainloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

In [8]:
val_set = MusicDataset(x_val,y_val)
validationloader = torch.utils.data.DataLoader(val_set, batch_size=1,
                                         shuffle=False, num_workers=2)

**DataLoader**

**For Wavenet Model (Ignore for now)**

In [12]:
from Models import Wavenet
Net = Wavenet.Wavenet(len(unique_notes),blocks=4,layers = 2)

**Checking the model**

In [ ]:
# Net = Sequence(len(unique_x),32);
# input = torch.zeros(1,x_tr[0].shape[0],dtype=torch.int32);
# input[0,:] = torch.IntTensor(x_tr[0])
# # input = input.permute(0,2,1)

# outputs = Net(input)
# print(outputs.shape)

In [13]:
# Net = Sequence(len(unique_x),32);

import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(Net.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
loss_history = []

**Use the line below when you want to change learning parameters in between**

In [ ]:
optimizer = torch.optim.Adam(Net.parameters(), lr=0.0001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

In [14]:
''' TRAINING THE MODEL '''
correct_preds = 0
total_preds = 0
for epoch in range(50): 
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        input , label = data
        optimizer.zero_grad()
        output = Net(input)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        total_preds += input.shape[0]
        correct_preds += torch.sum(torch.argmax(output,1) == label)

        running_loss += loss.item()
        if i % 200 == 199:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            loss_history.append(running_loss/200)
            running_loss = 0.0

    report ="correct predictions  : {} \n total predictions : {} \n Accuracy : {}".format(correct_preds,total_preds, correct_preds/total_preds *100)
    print(report)     
    correct_preds = 0
    total_preds = 0       

print('Finished Training')

correct predictions  : 90 
 total predictions : 942 
 Accuracy : 9.554140090942383
correct predictions  : 94 
 total predictions : 942 
 Accuracy : 9.978768348693848
correct predictions  : 126 
 total predictions : 942 
 Accuracy : 13.3757963180542
correct predictions  : 128 
 total predictions : 942 
 Accuracy : 13.58811092376709
correct predictions  : 137 
 total predictions : 942 
 Accuracy : 14.543524742126465
correct predictions  : 140 
 total predictions : 942 
 Accuracy : 14.8619966506958
correct predictions  : 144 
 total predictions : 942 
 Accuracy : 15.286624908447266
correct predictions  : 145 
 total predictions : 942 
 Accuracy : 15.392782211303711
correct predictions  : 144 
 total predictions : 942 
 Accuracy : 15.286624908447266
correct predictions  : 145 
 total predictions : 942 
 Accuracy : 15.392782211303711
correct predictions  : 144 
 total predictions : 942 
 Accuracy : 15.286624908447266
correct predictions  : 145 
 total predictions : 942 
 Accuracy : 15.39278

In [ ]:
plt.figure()
plt.plot(loss_history)
# plt.savefig('/content/drive/MyDrive/schubert/loss_history1.png')
# plt.savefig('/content/drive/MyDrive/schubert/sine_wavenet1.png')

In [ ]:
# path='/content/drive/MyDrive/schubert/sine_model.pth'
# path='/content/drive/MyDrive/schubert/lstmmodel1.pth'
path='/content/drive/MyDrive/schubert/sine_wavenet1.pth'
torch.save(Net.state_dict(), path)

In [ ]:
import random
ind = np.random.randint(0,len(unique_x)-1 )

random_sine_part = x_val[ind]
print(y_val[ind])
print(random_sine_part)

predictions = Net.predict(random_sine_part,steps=32)
predictions = np.hstack( predictions )
print(predictions)

import matplotlib.pyplot as plt
x_int_to_note = dict((number, note_) for number, note_ in enumerate(unique_x)) 
intial = [x_int_to_note[i] for i in random_sine_part]
pred = [x_int_to_note[i] for i in predictions]
plt.figure();
plt.plot(range(len(intial)),intial)
offset = len(random_sine_part)
plt.plot(range(offset,len(pred)+offset),pred)


In [ ]:
# path='/content/drive/MyDrive/schubert/sine_model.pth'
# path='/content/drive/MyDrive/schubert/lstmmodel1.pth'
path='/content/drive/MyDrive/schubert/sine_wavenet1.pth'
# Net = Sequence(len(unique_x),32);
Net = Wavenet(len(unique_x),blocks=4,layers = 2)
Net.load_state_dict(torch.load(path))

correct_preds = 0
total_preds = 0
for i, data in enumerate(validationloader, 0):
  input , label = data
  total_preds += 1
  break
  output = Net.predict(input,steps=1)
  if(output[0] == label.numpy()):
    correct_preds+=1 

report ="correct predictions  : {} \n total predictions : {} \n Accuracy : {}".format(correct_preds,total_preds, correct_preds/total_preds *100)
print(report)
